In [25]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date, datetime
import logging
import json
from tqdm import tqdm
from typing import List
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
import re
import time
from urllib.parse import quote
import warnings

In [ ]:
# Config together AI
TOGETHER_API_KEY = "87383fa10dfb16c491f9fb909a0c458ed5db1572bea1182ad941144fc16dee59"

API_URL = "https://api.together.xyz/v1/chat/completions"  # ← Use 'completions', NOT 'chat/completions'
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

HEADERS = {
    "Authorization": f"Bearer {TOGETHER_API_KEY}",
    "Content-Type": "application/json"
}
#Config Telegram
BOT_TOKEN = "8388119215:AAGPOOajcj0iyha5dsWa15yzjniNwyo2QF4" #scoresignalbot
JSON_STORAGE_FILE = "bot_messages.json"
my_chatID = ["5759737133"]  # yobra
#odd api key
# ODDS_API_KEY = "2da01db450a92274d1b05b64f9cb8669" # Not in use from openfootball/football.json
# RAPID_API_KEY = "163922200bmshb128a62208b4bf1p1ac241jsn5900dfbc6c1c" # rapid api not sustainable $USD per month

# Logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger("football-api-scraper")

TODAY = str(date.today())
os.chdir(r"C:\Users\bkangethe\Box\Main Folder\Analysis\My Projects\Data Sources")
OUTPUT_FILE = f"football_odds_{TODAY}.xlsx"

In [1]:
import pandas as pd
from league_data import LEAGUE_NAMES, LEAGUE_PATHS
print(f"\nLeague paths from openfootball: {len(LEAGUE_PATHS)}")
# See which leagues are in LEAGUE_PATHS but not in LEAGUE_NAMES
missing_mappings = set(LEAGUE_PATHS.keys()) - set(LEAGUE_NAMES.keys())
print(f"Leagues missing odds mappings: {missing_mappings}")

# See the actual names in LEAGUE_PATHS
print("\nAll leagues in LEAGUE_PATHS:")
for key, info in LEAGUE_PATHS.items():
    print(f"{key}: {info['name']}")

Loaded 16 valid league mappings from 23 total rows

League paths from openfootball: 16
Leagues missing odds mappings: set()

All leagues in LEAGUE_PATHS:
be.1: Belgian Pro League
en.1: Premier League
en.2: EFL Championship
en.3: EFL League One
en.4: EFL League Two
es.1: La Liga
es.2: La Liga 2
fr.1: Ligue 1
fr.2: Ligue 2
gr.1: Super League Greece
it.1: Serie A
it.2: Serie B
nl.1: Eredivisie
pt.1: Primeira Liga
sco.1: Scottish Premiership
tr.1: Süper Lig


In [18]:
from league_data import get_league_paths
# Get all league paths
league_paths = get_league_paths("2025-26")

# Generate actual GitHub raw URLs for each league
base_url = "https://raw.githubusercontent.com/openfootball/football.json/master/"

for league_key, info in league_paths.items():
    actual_url = base_url + info['path']
    print(f"{league_key} ({info['name']}): {actual_url}")

at.1 (Unknown League (at.1)): https://raw.githubusercontent.com/openfootball/football.json/master/2025-26/at.1.json
at.2 (Unknown League (at.2)): https://raw.githubusercontent.com/openfootball/football.json/master/2025-26/at.2.json
be.1 (Belgian Pro League): https://raw.githubusercontent.com/openfootball/football.json/master/2025-26/be.1.json
de.1 (Unknown League (de.1)): https://raw.githubusercontent.com/openfootball/football.json/master/2025-26/de.1.json
de.2 (Unknown League (de.2)): https://raw.githubusercontent.com/openfootball/football.json/master/2025-26/de.2.json
en.1 (Premier League): https://raw.githubusercontent.com/openfootball/football.json/master/2025-26/en.1.json
en.2 (EFL Championship): https://raw.githubusercontent.com/openfootball/football.json/master/2025-26/en.2.json
en.3 (EFL League One): https://raw.githubusercontent.com/openfootball/football.json/master/2025-26/en.3.json
en.4 (EFL League Two): https://raw.githubusercontent.com/openfootball/football.json/master/202

In [1]:
from load_fixture_data import load_fixtures
fixtures_df = load_fixtures()  # This loads your fixtures.csv file
print(f"✅ Loaded {len(fixtures_df)} fixtures from local CSV")
# Cell 3: Explore your data (this is where you do your analysis)
fixtures_df.head()



✅ Loaded 5788 fixtures from local CSV


,round,Date,time,home_team,away_team,home_score,away_score,league_key,league_name,season
0,Matchday 1,7/25/2025,20:45,Royal Antwerp FC,Union Saint-Gilloise,1.0,1.0,be.1,Belgian Pro League,2025-26
1,Matchday 1,7/26/2025,16:00,FCV Dender EH,Cercle Brugge,0.0,0.0,be.1,Belgian Pro League,2025-26
2,Matchday 1,7/26/2025,18:15,SV Zulte Waregem,KV Mechelen,1.0,1.0,be.1,Belgian Pro League,2025-26
3,Matchday 1,7/26/2025,20:45,RAA La Louviére,Standard Liège,0.0,2.0,be.1,Belgian Pro League,2025-26
4,Matchday 1,7/27/2025,13:30,RSC Anderlecht,KVC Westerlo,5.0,2.0,be.1,Belgian Pro League,2025-26


In [2]:
from league_data import fetch_league_paths
leagues = fetch_league_paths("2024-25")
print(f"Found {len(leagues)} leagues for 2023-24 season:\n")

Loaded 16 valid league mappings from 23 total rows
✅ Successfully loaded 16 league mappings
Found 26 leagues for 2023-24 season:



In [2]:
# Test fetching
from fetch_fixtures_live import fetch_fixtures
# Try a league with special characters
df = fetch_fixtures('tr.1')  # Belgian league has many special characters
print(df[['home_team', 'away_team']].head(12))

           home_team            away_team
0       Gaziantep FK          Galatasaray
1         Samsunspor       Gençlerbirliği
2        Antalyaspor         Kasımpaşa SK
3         Fenerbahçe           Alanyaspor
4           Eyüpspor            Konyaspor
5    Çaykur Rizespor              Göztepe
6   Fatih Karagümrük  İstanbul Başakşehir
7        Kayserispor             Beşiktaş
8        Trabzonspor          Kocaelispor
9        Galatasaray     Fatih Karagümrük
10       Kocaelispor           Samsunspor
11        Alanyaspor      Çaykur Rizespor


In [1]:
# In your main pipeline
from load_fixture_data import load_fixtures
from data_utils import validate_team_mapping
import pandas as pd

# Load data
fixtures_df = load_fixtures()
mapping_df = pd.read_csv("team_mapping.csv")  # or however you load it

# Validate mappings
unmapped = validate_team_mapping(fixtures_df, mapping_df)

✅ All fixture teams have mappings!


In [2]:
df = pd.read_csv("cleaned_historical_data.csv")
print(f"Total matches: {len(df):,}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"Leagues: {df['league_code'].unique()}")
print(f"Teams: {df['HomeTeam'].nunique()} unique teams")
print(f"Bet365 coverage: {df['B365H'].notna().mean():.1%} of matches")

Total matches: 103,535
Date range: 1/1/2011 to 9/9/2024
Leagues: ['E2' 'E3' 'E0' 'SC0' 'E1' 'F1' 'F2' 'D2' 'N1' 'P1' 'T1' 'I2' 'B1' 'SP2'
 'D1' 'I1' 'SC1' 'G1' 'SP1']
Teams: 726 unique teams
Bet365 coverage: 99.8% of matches


C:\Users\bkangethe\AppData\Local\Temp\ipykernel_18760\2805102608.py:3: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("cleaned_historical_data.csv")


In [5]:
from load_clean_historical_data import load_clean_historical_data
modelling_df = load_clean_historical_data()
print("\n🔍 Data Quality Check:")
print(f"Matches with missing results: {(modelling_df['FTHG'].isna() | modelling_df['FTAG'].isna()).sum()}")
print(f"Matches with missing Bet365 odds: {modelling_df['B365H'].isna().sum()}")
print(f"Top leagues: {modelling_df['league_code'].value_counts().head(5).to_dict()}")



🔍 Data Quality Check:
Matches with missing results: 0
Matches with missing Bet365 odds: 235
Top leagues: {'E1': 8351, 'E3': 8275, 'E2': 8231, 'SP2': 6996, 'I2': 6358}


In [1]:
import requests
TOKEN = "8388119215:AAGPOOajcj0iyha5dsWa15yzjniNwyo2QF4"
WEBHOOK_URL = "https://your-web-service.up.railway.app/webhook"
response = requests.get(f"https://api.telegram.org/bot{TOKEN}/getWebhookInfo")
print(response.json())

{'ok': True, 'result': {'url': 'https://unhasted-clotty-shaunta.ngrok-free.dev/webhook', 'has_custom_certificate': False, 'pending_update_count': 3, 'last_error_date': 1759006839, 'last_error_message': 'Wrong response from the webhook: 404 Not Found', 'max_connections': 40, 'ip_address': '3.124.142.205'}}


In [1]:
import requests

BOT_TOKEN = "8388119215:AAGPOOajcj0iyha5dsWa15yzjniNwyo2QF4"
WEBHOOK_URL = "https://your-webhook-service.up.railway.app/webhook"

# Set webhook
response = requests.get(f"https://api.telegram.org/bot{BOT_TOKEN}/setWebhook?url={WEBHOOK_URL}")
print(response.json())

{'ok': False, 'error_code': 404, 'description': 'Not Found'}


In [3]:

import requests
requests.post("https://scoresignalbot-production.up.railway.app/webhook", json={})

<Response [400]>

In [9]:
import requests
requests.post("https://scoresignalbot-production.up.railway.app/webhook", json={})  # ← This sends valid empty JSON

<Response [400]>

In [14]:
python -c "from src.league_data import LEAGUE_PATHS; print(f'Leagues: {len(LEAGUE_PATHS)}')"

SyntaxError: invalid syntax (1547347324.py, line 1)

In [11]:
requests.get(f"https://api.telegram.org/bot{"8388119215:AAGPOOajcj0iyha5dsWa15yzjniNwyo2QF4"}/setWebhook?url={WEBHOOK_URL}")

<Response [200]>

In [ ]:
def fetch_expert_tips_ml(odds_df: pd.DataFrame, recent_results_df_all: pd.DataFrame, 
                        strong_threshold: float = 0.15, weight_odds: float = 0.8) -> pd.DataFrame:
    """
    Enhanced expert tips with better feature engineering and model architecture
    """
    # --- Enhanced Feature Engineering ---
    hist_df = recent_results_df_all.copy()
    hist_df['result'] = np.where(hist_df['home_score'] > hist_df['away_score'], 'home',
                                 np.where(hist_df['home_score'] < hist_df['away_score'], 'away', 'draw'))
    
    # Add default odds columns to historical data since they don't exist
    hist_df['home_odds'] = 2.0  # Default value
    hist_df['draw_odds'] = 3.0  # Default value  
    hist_df['away_odds'] = 2.5  # Default value

    # Advanced team form calculation
    def advanced_team_form(team, league=None, max_matches=10):
        """Calculate advanced form metrics with league weighting"""
        mask = (hist_df['home_team'] == team) | (hist_df['away_team'] == team)
        if league and 'league' in hist_df.columns:
            mask &= (hist_df['league'] == league)
        
        team_results = hist_df[mask].sort_values('date', ascending=False).head(max_matches)
        
        if team_results.empty:
            return {'win_rate': 0.5, 'goal_diff': 0, 'form_strength': 0.5, 'recent_goals': 1.0}
        
        wins, draws, losses = 0, 0, 0
        goals_for, goals_against = 0, 0
        
        for _, match in team_results.iterrows():
            if match['home_team'] == team:
                goals_for += match['home_score']
                goals_against += match['away_score']
                if match['home_score'] > match['away_score']:
                    wins += 1
                elif match['home_score'] == match['away_score']:
                    draws += 1
                else:
                    losses += 1
            else:
                goals_for += match['away_score']
                goals_against += match['home_score']
                if match['away_score'] > match['home_score']:
                    wins += 1
                elif match['away_score'] == match['home_score']:
                    draws += 1
                else:
                    losses += 1
        
        total_matches = wins + draws + losses
        win_rate = wins / total_matches if total_matches > 0 else 0.5
        goal_diff = (goals_for - goals_against) / total_matches if total_matches > 0 else 0
        form_strength = (wins * 1 + draws * 0.5) / total_matches if total_matches > 0 else 0.5
        recent_goals = goals_for / total_matches if total_matches > 0 else 1.0
        
        return {
            'win_rate': win_rate,
            'goal_diff': goal_diff,
            'form_strength': form_strength,
            'recent_goals': recent_goals
        }

    # --- Enhanced Model Training ---
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.preprocessing import StandardScaler
    
    # Prepare training data with enhanced features
    X_train = []
    y_train = []
    
    for _, match in hist_df.iterrows():
        # Use league if available, otherwise None
        league = match.get('league') if 'league' in hist_df.columns else None
        
        home_form = advanced_team_form(match['home_team'], league)
        away_form = advanced_team_form(match['away_team'], league)
        
        features = [
            match['home_odds'], match['draw_odds'], match['away_odds'],
            home_form['win_rate'], home_form['goal_diff'], home_form['form_strength'],
            away_form['win_rate'], away_form['goal_diff'], away_form['form_strength'],
            home_form['recent_goals'] - away_form['recent_goals']  # Goal difference trend
        ]
        
        X_train.append(features)
        y_train.append(match['result'])
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    # Use Gradient Boosting for better performance
    model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, 
                                     max_depth=4, random_state=42)
    model.fit(X_train_scaled, y_train)

    # --- Predict on upcoming matches ---
    df = odds_df.copy()
    df = df.rename(columns={'home': 'home_team', 'away': 'away_team'})
    
    home_win_prob, draw_prob, away_win_prob, expert_tip, strong_tip = [], [], [], [], []
    
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Generating Expert Tips"):
        home, away = row['home_team'], row['away_team']
        h_odds, d_odds, a_odds = row['home_odds'], row['draw_odds'], row['away_odds']
        
        # Get league if available
        league = row.get('league') if 'league' in df.columns else None
        
        # Get team form
        home_form = advanced_team_form(home, league)
        away_form = advanced_team_form(away, league)
        
        # Prepare features for prediction
        features = [
            h_odds, d_odds, a_odds,
            home_form['win_rate'], home_form['goal_diff'], home_form['form_strength'],
            away_form['win_rate'], away_form['goal_diff'], away_form['form_strength'],
            home_form['recent_goals'] - away_form['recent_goals']
        ]
        
        # Scale and predict
        features_scaled = scaler.transform([features])
        probs = model.predict_proba(features_scaled)[0]
        classes = model.classes_
        prob_dict = dict(zip(classes, probs))
        
        # Store probabilities
        home_win_prob.append(prob_dict.get('home', 0))
        draw_prob.append(prob_dict.get('draw', 0))
        away_win_prob.append(prob_dict.get('away', 0))
        
        # Expert tip
        tip = max(prob_dict, key=prob_dict.get)
        expert_tip.append(tip)
        
        # Strong tip
        sorted_probs = sorted(prob_dict.values(), reverse=True)
        strong_tip.append(sorted_probs[0] - sorted_probs[1] >= strong_threshold)
    
    # Add results to DataFrame
    df['home_win_prob'] = home_win_prob
    df['draw_prob'] = draw_prob
    df['away_win_prob'] = away_win_prob
    df['expert_tip'] = expert_tip
    df['strong_tip'] = strong_tip
    
    return df

In [ ]:
def enrich_with_insights(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add analytical flags to enhance LLM context: favorites, value bets, tossups.
    """
    df = df.copy()

    # Win probability thresholds
    df['home_win_prob_pct'] = (df['home_win_prob'] * 100).round()
    df['draw_prob_pct'] = (df['draw_prob'] * 100).round()
    df['away_win_prob_pct'] = (df['away_win_prob'] * 100).round()

    # Team strength
    df['is_home_favorite'] = df['home_win_prob'] > 0.55
    df['is_away_favorite'] = df['away_win_prob'] > 0.55
    df['is_tossup'] = (abs(df['home_win_prob'] - df['away_win_prob']) < 0.15) & (df['draw_prob'] < 0.35)

    # Value betting flags
    df['has_high_draw_odds'] = df['draw_odds'] > 3.4
    df['home_good_value'] = (df['home_odds'] >= 2.4) & (df['home_win_prob'] > 0.4)
    df['away_underdog_value'] = (df['away_odds'] >= 3.0) & (df['away_win_prob'] > 0.3)

    # Strong tip (from earlier logic)
    df['strong_ml_tip'] = df['strong_tip']

    return df

In [ ]:
# Example usage in notebook
df_with_tips = fetch_expert_tips_ml(odds_df, recent_results_df_all, strong_threshold=0.2)
df_enriched = enrich_with_insights(df_with_tips)

In [ ]:
def build_enhanced_prompt(row) -> str:
    """
    Build a rich, contextual prompt for the LLM using ML + odds insights.
    """
    home, away = row['home_team'], row['away_team']
    league = row.get('league', 'a league')

    # Probabilities as percentages
    h_prob = row['home_win_prob']
    d_prob = row['draw_prob']
    a_prob = row['away_win_prob']

    # --- Momentum & Dominance ---
    if h_prob > 0.6:
        momentum = f"{home} are strong favorites based on form and consistent performance."
    elif a_prob > 0.6:
        momentum = f"{away} appear superior on paper and should dominate despite being away."
    elif abs(h_prob - a_prob) < 0.15:
        momentum = "This is a tightly contested match between evenly matched teams."
    else:
        momentum = f"{home} hold a slight edge at home, but {away} have shown resilience recently."

    # --- Value Detection ---
    value_lines = []
    if row['has_high_draw_odds'] and d_prob > 0.28:
        value_lines.append(f"The draw at {row['draw_odds']} offers solid value given its {d_prob:.0%} likelihood.")
    if row['home_good_value']:
        value_lines.append(f"{home} at {row['home_odds']} represents promising value against the odds.")
    if row['away_underdog_value']:
        value_lines.append(f"{away} could be a smart longshot play at {row['away_odds']}.")
    
    value_tip = " ".join(value_lines) if value_lines else ""

    # --- Expert Tip Context ---
    expert_call = row['expert_tip'].upper()
    if expert_call == 'HOME':
        expert_reason = f"{home} are favored by both model and market."
    elif expert_call == 'AWAY':
        expert_reason = f"{away} are backed by strong away-form metrics."
    elif expert_call == 'DRAW':
        expert_reason = f"The model sees this as a tactical stalemate with high draw probability."
    else:
        expert_reason = f"This match lacks a clear favorite according to analytics."

    # --- Final Prompt ---
    prompt = f"""
You are a top-tier football betting analyst writing for a premium tips platform.
Write a **confident, concise 1–2 sentence summary** predicting the outcome of this match.

Guidelines:
- Be authoritative and specific. No disclaimers like "unpredictable" or "can go either way".
- Mention team strength, momentum, or value in the odds.
- Use active language: "will", "should", "look set to".
- NEVER hedge. ALWAYS pick a side unless the draw is strongly supported.

Match: {home} vs {away}
League: {league}
Odds (H/D/A): {row['home_odds']} / {row['draw_odds']} / {row['away_odds']}
Win Probabilities: {h_prob:.0%} / {d_prob:.0%} / {a_prob:.0%}
Expert Tip: {expert_call}

Context:
{momentum}
{value_tip}
{expert_reason}

Now write the final summary:
"""
    return prompt.strip()

In [ ]:
def is_generic(text: str) -> bool:
    if not text or len(text.strip()) < 5:
        return True
    text_lower = text.lower()
    bad_phrases = ['unpredictable', 'hard to call', 'either way', 'no clear favorite', 
                   'could go either way', 'uncertain', 'caution', 'avoid']
    return any(phrase in text_lower for phrase in bad_phrases)

def fallback_summary(row):
    tip = row['expert_tip'].upper()
    if tip == 'HOME':
        return f"{row['home_team']} are favored based on form and odds."
    elif tip == 'AWAY':
        return f"{row['away_team']} holds an edge on the road."
    elif tip == 'DRAW':
        return f"Model suggests a draw; consider value at current odds."
    else:
        return f"Match analysis shows no strong lean."

def refine_with_fallback(row, original=""):
    tip = row['expert_tip']
    h, a = row['home_team'], row['away_team']
    if tip == 'home':
        return f"{h} should edge this contest based on superior form and home advantage."
    elif tip == 'away':
        return f"{a} looks capable of snatching three points despite being away."
    elif tip == 'draw':
        return f"Expect a tight tactical battle ending in a draw."
    else:
        return f"{h} vs {a}: too close to call, slight nod to home side."

In [ ]:
# Configure Your API (Together AI Example) ---
def add_llm_summaries(
    odds_df: pd.DataFrame,
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    api_url="https://api.together.xyz/v1/chat/completions",  # ← Using chat endpoint
    headers=HEADERS,
    max_retries=3,
    delay_per_call=0.5
) -> pd.DataFrame:
    """
    Generate LLM summaries with clean progress tracking and proper response handling.
    Uses chat/completions → parses 'message.content'.
    """
    summaries = []

    # Single progress bar with clear updates
    for _, row in tqdm(odds_df.iterrows(), total=len(odds_df), desc="🧠 Generating Expert Summaries", unit="match"):
        summary_text = ""

        try:
            prompt = build_enhanced_prompt(row)
        except Exception as e:
            print(f"Prompt build failed for {row['home_team']} vs {row['away_team']}: {e}")
            summary_text = fallback_summary(row)
            summaries.append(summary_text)
            time.sleep(delay_per_call)
            continue

        success = False
        for attempt in range(max_retries + 1):
            try:
                payload = {
                    "model": model_name,
                    "messages": [{"role": "user", "content": prompt}],
                    "max_tokens": 90,
                    "temperature": 0.3,
                    "top_p": 0.9,
                    "repetition_penalty": 1.05,
                    "stop": ["\n\n", "Match:", "League:", "User:", "Assistant:"]
                }

                response = requests.post(api_url, headers=headers, json=payload, timeout=30)
                
                if response.status_code == 200:
                    data = response.json()
                    
                    # ✅ Correct key for /chat/completions
                    content = data["choices"][0]["message"]["content"].strip()
                    
                    # Clean up: take only first 1–2 sentences
                    sentences = [s.strip() for s in content.split('. ') if s.strip()]
                    summary_text = '. '.join(sentences[:2]) + '.' if sentences else fallback_summary(row)
                    
                    success = True
                    break
                else:
                    print(f"❌ HTTP {response.status_code} on attempt {attempt + 1}: {response.text[:100]}...")
                    time.sleep(2 ** attempt)  # Exponential backoff

            except KeyError as e:
                print(f"🔑 Parse error (missing key {e}) - retrying...")
                time.sleep(1.5)
            except Exception as e:
                if attempt >= max_retries:
                    print(f"🛑 Failed after retries: {row['home_team']} vs {row['away_team']} | Error: {e}")
                else:
                    time.sleep(2 ** attempt)

        if not success:
            summary_text = fallback_summary(row)

        # Final quality filter
        if is_generic(summary_text):
            summary_text = refine_with_fallback(row, summary_text)

        summaries.append(summary_text)
        time.sleep(delay_per_call)

    # Add to DataFrame
    result_df = odds_df.copy()
    result_df['tip_summary'] = summaries
    return result_df

In [ ]:
test_prompt = build_enhanced_prompt(df_enriched.iloc[0])

payload = {
    "model": MODEL_NAME,
    "messages": [{"role": "user", "content": test_prompt}],
    "max_tokens": 90,
    "temperature": 0.3,
    "top_p": 0.9,
    "repetition_penalty": 1.05,
    "stop": ["\n\n", "Match:", "User:", "Assistant:"]
}

resp = requests.post(API_URL, headers=HEADERS, json=payload)
print("Status:", resp.status_code)
if resp.status_code == 200:
    print("Success:\n", resp.json()["choices"][0]["message"]["content"].strip())
else:
    print("Error:\n", resp.text)

In [ ]:
# workflow
df_tips = fetch_expert_tips_ml(odds_df, recent_results_df_all, strong_threshold=0.2)
df_enriched = enrich_with_insights(df_tips)
df_final = add_llm_summaries(df_enriched, delay_per_call=0.5)
df_final['tip_source'] = df_final['strong_tip'].map({True: 'strong_ml', False: 'weak_ml'})
cols = [
    'home_team', 'away_team', 'league',
    'home_odds', 'draw_odds', 'away_odds',
    'home_win_prob', 'draw_prob', 'away_win_prob',
    'expert_tip', 'strong_tip', 'tip_source',  # if you added ensemble earlier
    'tip_summary'
]

display(df_final[cols].head(2))

In [ ]:
# Remove timezone info from all DataFrames before saving to Excel
def remove_timezone_from_df(df):
    """Remove timezone info from datetime columns in a DataFrame"""
    if df is None or df.empty:
        return df
    
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            df[col] = df[col].dt.tz_localize(None)  # Remove timezone info
    return df

In [ ]:
# File configuration
TODAY = datetime.today().strftime("%Y-%m-%d")
OUTPUT_FILE = f"football_data_{TODAY}.xlsx"

# Create a dictionary of all DataFrames to clean
dataframes_to_clean = {
    "Fixtures": fixtures_df,
    "Odds": odds_df,
    "Recent Results": recent_results_df_all,
    "Expert Tips": df_enriched,
    "LLM Summaries": df_final
}

# Clean all DataFrames in one go using dictionary comprehension
cleaned_dataframes = {
    sheet_name: remove_timezone_from_df(df.copy()) 
    for sheet_name, df in dataframes_to_clean.items()
}

# Create main metadata DataFrame
metadata = {
    "Generated On": [datetime.now().replace(tzinfo=None)],
    "Leagues Included": [", ".join(all_leagues)],
    "Total Fixtures": [len(fixtures_df)],
    "Total Odds": [len(odds_df)],
    "Total Recent Results": [len(recent_results_df_all)],
    "Total Matches with Tips": [len(df_final)],
    "Total Matches with LLM Summaries": [len(df_final)]
}
metadata_df = pd.DataFrame(metadata)

# Create soccer keys DataFrame
if not df_soccer.empty:
    soccer_keys_list = [f"{row['key']} ({row['title']})" for _, row in df_soccer.iterrows()]
else:
    soccer_keys_list = ["No soccer keys data"]

soccer_keys_df = pd.DataFrame({
    "Available Soccer Keys": soccer_keys_list
})

# Save to Excel
with pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl") as writer:
    # Write main DataFrames
    for sheet_name, df_clean in cleaned_dataframes.items():
        df_clean.to_excel(writer, sheet_name=sheet_name, index=False)
    
    # Write metadata section
    metadata_df.to_excel(writer, sheet_name="Metadata", index=False)
    
    # Write soccer keys section with empty row separation
    start_row = len(metadata_df) + 2  # +2 for header row and one empty row
    soccer_keys_df.to_excel(writer, sheet_name="Metadata", index=False, 
                          startrow=start_row, header=True)

print(f"All data saved successfully to {OUTPUT_FILE}")

In [ ]:
# Strong tips within the next 48 hours
def filter_strong_tips(df_final: pd.DataFrame, max_hours_ahead=48) -> pd.DataFrame:
    """
    Filter strong ML tips where:
    - tip_source == 'strong_ml'
    - match has not started
    - match starts within the next 2 days (48 hours)
    Returns cleaned DataFrame with odds and formatted time.
    """
    # Step 1: Filter strong tips
    if df_final.empty:
        return df_final

    strong_mask = df_final['tip_source'] == 'strong_ml'
    strong_tips_df = df_final[strong_mask].copy()
    
    if strong_tips_df.empty:
        return strong_tips_df

    # Ensure datetime column exists and is datetime type
    if 'datetime_local' not in strong_tips_df.columns:
        raise KeyError("Column 'datetime_local' not found in DataFrame.")
        
    if not pd.api.types.is_datetime64_any_dtype(strong_tips_df['datetime_local']):
        raise TypeError("Column 'datetime_local' must be a datetime type.")

    # Remove timezone if present (ensure naive)
    if strong_tips_df['datetime_local'].dt.tz is not None:
        print("Removing timezone info from datetime_local...")
        dt_local = strong_tips_df['datetime_local'].dt.tz_localize(None)
    else:
        dt_local = strong_tips_df['datetime_local']

    # Use Africa/Nairobi as reference time context
    try:
        now_naive = pd.Timestamp.now(tz='Africa/Nairobi').tz_localize(None)
    except Exception:
        print("Warning: Fallback to system time.")
        now_naive = pd.Timestamp.now()

    # Define time window: now to now + 2 days
    two_days_ahead = now_naive + pd.Timedelta(days=2)

    # Create mask: match hasn't started AND is within next 2 days
    upcoming_mask = (dt_local > now_naive) & (dt_local <= two_days_ahead)
    upcoming_tips_df = strong_tips_df[upcoming_mask].copy()

    if upcoming_tips_df.empty:
        return upcoming_tips_df

    # Column mapping for output
    column_mapping = {
        'league': 'league',
        'datetime_local': 'datetime_local',
        'home_team': 'home',
        'away_team': 'away',
        'home_odds': 'home_odds',
        'draw_odds': 'draw_odds',
        'away_odds': 'away_odds',
        'expert_tip': 'expert_tip',
        'tip_summary': 'tip_summary'
    }

    missing_cols = [col for col in column_mapping.keys() if col not in upcoming_tips_df.columns]
    if missing_cols:
        raise KeyError(f"Missing required columns: {missing_cols}")

    result = upcoming_tips_df[list(column_mapping.keys())].rename(columns=column_mapping)

    # Format datetime as string for display
    result['datetime_local'] = result['datetime_local'].dt.strftime('%Y-%m-%d %H:%M')

    return result

In [ ]:
# Manage message history
def load_message_history(storage_file=JSON_STORAGE_FILE):
    """Load message history from JSON file"""
    try:
        with open(storage_file, 'r') as f:
            return json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        return {}

def save_message_history(history, storage_file=JSON_STORAGE_FILE):
    """Save message history to JSON file"""
    with open(storage_file, 'w') as f:
        json.dump(history, f, indent=2)

def track_message(chat_id, message_id, storage_file=JSON_STORAGE_FILE):
    """Track a message ID for a specific chat in persistent storage"""
    history = load_message_history(storage_file)
    
    if str(chat_id) not in history:
        history[str(chat_id)] = []
    
    # Add new message ID and keep only unique IDs
    if message_id not in history[str(chat_id)]:
        history[str(chat_id)].append(message_id)
    
    # Optional: Limit to last 200 messages per chat to avoid file bloat
    history[str(chat_id)] = history[str(chat_id)][-200:]
    
    save_message_history(history, storage_file)


In [7]:
# bot/check_structure.py
import os
import sys

def check_structure():
    print("🔍 Checking directory structure...")
    
    # Current working directory
    print(f"📁 Current working directory: {os.getcwd()}")
    
    # Where this script is located
    script_dir = os.path.dirname(os.path.abspath(__file__))
    print(f"📁 Script directory: {script_dir}")
    
    # List files in script directory
    print(f"📂 Files in bot directory:")
    for file in os.listdir(script_dir):
        print(f"   - {file}")
    
    # Check if prediction_main.py exists
    prediction_path = os.path.join(script_dir, "prediction_main.py")
    print(f"🔎 Looking for prediction_main.py at: {prediction_path}")
    print(f"✅ File exists: {os.path.exists(prediction_path)}")
    
    # Check Python path
    print(f"🐍 Python executable: {sys.executable}")
    print(f"📦 Python path: {sys.path}")

In [ ]:
# send messages
def send_telegram_message(message, chat_ids, bot_token, track_message_id=True, storage_file=JSON_STORAGE_FILE):
    """
    Helper function to send a single Telegram message with optional tracking
    
    Parameters:
    message (str): The message to send
    chat_ids (list): List of chat IDs to send to
    bot_token (str): Bot token
    track_message_id (bool): Whether to track the message ID for later deletion
    storage_file (str): Path to the JSON storage file
    
    Returns:
    bool: True if at least one message was sent successfully
    """
    success_count = 0
    
    for chat_id in chat_ids:
        try:
            url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
            payload = {
                'chat_id': chat_id,
                'text': message,
                'parse_mode': 'Markdown',
                'disable_web_page_preview': True
            }
            
            response = requests.post(url, data=payload)
            if response.status_code == 200:
                success_count += 1
                
                # Track the message ID if requested
                if track_message_id:
                    message_id = response.json()['result']['message_id']
                    track_message(chat_id, message_id, storage_file)
                    
            else:
                print(f"❌ Failed to send message to {chat_id}: {response.text}")
                
        except Exception as e:
            print(f"❌ Error sending message to {chat_id}: {e}")
    
    return success_count > 0


In [ ]:
# send strong tips
def send_strong_tips_telegram_multiple_users(strong_tips_df, chat_ids, bot_token, track_messages=True, storage_file=JSON_STORAGE_FILE):
    """
    Send strong tips to multiple Telegram users with message chunking and tracking
    
    Parameters:
    strong_tips_df (DataFrame): DataFrame with strong tips data
    chat_ids (list): List of chat IDs to send to
    bot_token (str): Bot token
    track_messages (bool): Whether to track message IDs for later deletion
    storage_file (str): Path to the JSON storage file
    
    Returns:
    int: Total number of successful message sends
    """
    if strong_tips_df.empty:
        message = "⚽ No strong tips found today! ⚽"
        send_count = 1 if send_telegram_message(message, chat_ids, bot_token, track_messages, storage_file) else 0
        return send_count
    
    total_success_count = 0
    
    for chat_id in chat_ids:
        chat_success_count = 0
        
        try:
            # Create message header
            header = f"⚽ *STRONG BETTING TIPS* ⚽\n\n"
            header += f"Found {len(strong_tips_df)} matches with strong tips:\n\n"
            
            # ✅ Add payment information
            header += "🔐 *For advanced modelling tips, support via MPESA:* \n"
            header += "📌 *TILL: 9105695*\n"
            header += "📱 *Mobile:* +254722586447\n\n"  # Replace 07XXXXXX with actual number
            header += "Thank you for your support! 💙\n\n"
            
            # Send header first
            if send_telegram_message(header, [chat_id], bot_token, track_messages, storage_file):
                chat_success_count += 1
            
            # Send each match as separate message
            for _, row in strong_tips_df.iterrows():
                match_message = f"🏆 *{row['league']}*\n"
                match_message += f"⏰ {row['datetime_local']}\n"
                match_message += f"🏠 {row['home']} vs {row['away']}\n"
                
                # ✅ Correctly show decimal odds
                match_message += f"🔢 Odds: {row['home_odds']:.2f} | {row['draw_odds']:.2f} | {row['away_odds']:.2f}\n"
                
                # Show expert tip
                tip_emoji = "🟢 HOME" if row['expert_tip'] == 'home' else \
                           ("🟡 DRAW" if row['expert_tip'] == 'draw' else "🔵 AWAY")
                match_message += f"💡 Tip: {tip_emoji} ({row['expert_tip'].upper()})\n"
                
                # Truncate summary if too long
                summary = str(row['tip_summary'])
                if len(summary) > 200:
                    match_message += f"📝 Summary: {summary[:200]}...\n"
                else:
                    match_message += f"📝 Summary: {summary}\n"
                
                match_message += "─" * 30
                
                # Send individual match message
                if send_telegram_message(match_message, [chat_id], bot_token, track_messages, storage_file):
                    chat_success_count += 1
            
            print(f"✅ {chat_success_count} messages sent to chat ID: {chat_id}")
            total_success_count += chat_success_count
                
        except Exception as e:
            print(f"❌ Error sending to {chat_id}: {e}")
    
    print(f"\n📊 Summary: Total messages sent: {total_success_count}")
    return total_success_count

In [ ]:
# get chat IDS
def get_chat_ids(bot_token):
    """Get all chat IDs that have interacted with your bot"""
    url = f"https://api.telegram.org/bot{bot_token}/getUpdates"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            chat_ids = set()
            
            for update in data.get('result', []):
                if 'message' in update and 'chat' in update['message']:
                    chat_ids.add(update['message']['chat']['id'])
            
            print("Found chat IDs:", list(chat_ids))
            return list(chat_ids)
        else:
            print("Error getting updates:", response.text)
            return []
    except Exception as e:
        print("Error:", e)
        return []

def clear_bot_messages(chat_id=None, bot_token=BOT_TOKEN, storage_file=JSON_STORAGE_FILE):
    """
    Clear all tracked bot messages for a specific chat or all chats.
    
    Parameters:
    chat_id (int/str): Specific chat ID to clear. If None, clears all chats.
    bot_token (str): Bot token to use for deletion
    storage_file (str): Path to the JSON storage file
    """
    history = load_message_history(storage_file)
    
    if not history:
        print("No message history found to clear.")
        return 0
    
    chats_to_clear = [str(chat_id)] if chat_id else list(history.keys())
    total_deleted = 0
    
    for current_chat_id in chats_to_clear:
        if current_chat_id not in history:
            print(f"Chat ID {current_chat_id} not found in history.")
            continue
            
        message_ids = history[current_chat_id]
        deleted_in_chat = 0
        
        print(f"\nClearing {len(message_ids)} messages in chat {current_chat_id}...")
        
        for msg_id in message_ids:
            url = f"https://api.telegram.org/bot{bot_token}/deleteMessage"
            payload = {
                'chat_id': current_chat_id,
                'message_id': msg_id
            }
            
            try:
                response = requests.post(url, data=payload)
                if response.status_code == 200:
                    deleted_in_chat += 1
                    total_deleted += 1
                    print(f"✓ Deleted message {msg_id}")
                else:
                    print(f"✗ Failed to delete {msg_id}: {response.text}")
                
                # Add a small delay to avoid hitting rate limits
                time.sleep(0.1)
                
            except Exception as e:
                print(f"✗ Error deleting {msg_id}: {e}")
        
        # Clear the history for this chat after successful deletion
        history[current_chat_id] = []
        print(f"Cleared {deleted_in_chat} messages from chat {current_chat_id}")
    
    # Save the updated history (with cleared chats)
    save_message_history(history, storage_file)
    print(f"\n🎉 Total messages deleted: {total_deleted}")
    return total_deleted

def show_message_stats(storage_file=JSON_STORAGE_FILE):
    """Show statistics about tracked messages"""
    history = load_message_history(storage_file)
    total_messages = 0
    
    print("📊 Message Tracking Statistics:")
    print("-" * 40)
    
    for chat_id, messages in history.items():
        print(f"Chat {chat_id}: {len(messages)} messages")
        total_messages += len(messages)
    
    print("-" * 40)
    print(f"Total chats: {len(history)}")
    print(f"Total messages: {total_messages}")
    return len(history), total_messages

In [ ]:
print("Getting chat IDs...")
chat_ids = get_chat_ids(BOT_TOKEN)

# Print the number of participants
if chat_ids:
    print(f"✅ Number of participants: {len(chat_ids)}")
else:
    print("❌ No participants found. Chat ID list is empty.")

In [ ]:
# Filter the strong tips
df_final = remove_timezone_from_df(df_final) 
strong_tips_filtered = filter_strong_tips(df_final, max_hours_ahead=48)  # 2-day lookahead
print("\nSending strong tips with tracking...")
send_count = send_strong_tips_telegram_multiple_users(
    strong_tips_filtered, 
    chat_ids, 
    BOT_TOKEN,
    track_messages=True  # Set to False if you don't want to track these messages
)
print(f"Sent {send_count} messages")

# Show current message statistics
print("\nCurrent stats:")
show_message_stats()

In [ ]:
# Clearing messages
# Clear messages from a specific chat
# clear_bot_messages(chat_id=123456789, bot_token=BOT_TOKEN)

# Clear messages from ALL chats (use with caution!)
#print("\nClearing all messages...")
#deleted_count = clear_bot_messages(bot_token=BOT_TOKEN)
#print(f"Deleted {deleted_count} messages")

# Send a message without tracking (for one-time notifications)
# send_telegram_message("This message won't be tracked", chat_ids, BOT_TOKEN, track_message_id=False)